This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using Pyomo.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pyo
import pyomo.dae as dae

This example has a series reaction $A \rightarrow B \rightarrow C$. The dataset consists of measures concentrations of A, B and C over time. The goal is to estimate the rate constants $k_1$ and $k_2$ for the two reactions.

In [ ]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()

In [ ]:
# Convert data to a list of dictionaries
data = [{'ca_meas': {k:v for (k, v) in zip(data_df.t, data_df.ca)},
    'cb_meas': {k:v for (k, v) in zip(data_df.t, data_df.cb)},
    'cc_meas': {k:v for (k, v) in zip(data_df.t, data_df.cc)} }]

In [68]:
#
# Define the model 
#

m = pyo.ConcreteModel()

rxns = [0, 1]
m.k = pyo.Var(rxns, initialize = 0.5, bounds = (1e-4, 10))
    
k = [0]

def genblock(b, k):
    b.dummy = pyo.Param(initialize = 1)
m.b = pyo.Block(k, rule = genblock)

In [60]:
def ABC_model(b, data, disctype, sim):
    
    ca_meas = data['ca_meas']
    cb_meas = data['cb_meas']
    cc_meas = data['cc_meas']
    
    species = ['a', 'b', 'c']
    id_species = {'a': 0, 'b': 1, 'c': 2}
    
    S = np.array([[-1.0, 1.0, 0.0],
                  [0.0, -1.0, 1.0]
                 ])
    
    meas_t = list(ca_meas.keys())
           
    c0 = {'a': 1.0, 'b': 0.0, 'c': 0.0}
    
    def _c_init_rule(b, t, j):
        return c0[j]
    
    b.errsq = pyo.Var(within = pyo.NonNegativeReals)
    b.time = dae.ContinuousSet(bounds = (0.0, 5.0), initialize = meas_t)
    b.c = pyo.Var(b.time, species, initialize = _c_init_rule, bounds = (0, 1))
    
    b.dc = dae.DerivativeVar(b.c, wrt = b.time)
    
    def _dcrate(b, t, j):
        
        nrxn = S.shape[0]
        nspec = S.shape[1]
        rrate = {}
        for i in range(nrxn):
            rrate[i] = m.k[i]
            for j2 in range(nspec):
                if S[i, j2] < 0:
                    rrate[i] = rrate[i] * b.c[t, species[j2]]
        
        if t == 0:
            return pyo.Constraint.Skip
        else:
            return b.dc[t, j] == sum(S[i, id_species[j]] * rrate[i] for i in rxns)

    def _dcrate_sim(b, t, j):
        nrxn = S.shape[0]
        nspec = S.shape[1]
        rrate = {}
        for i in range(nrxn):
            rrate[i] = m.k[i]
            for j2 in range(nspec):
                if S[i, j2] < 0:
                    rrate[i] = rrate[i] * b.c[t, species[j2]]
        return b.dc[t, j] == sum(S[i, id_species[j]] * rrate[i] for i in rxns)
    
    if sim == 0:
        b.dcrate = pyo.Constraint(b.time, species, rule = _dcrate)
    else:
        b.dcrate = pyo.Constraint(b.time, species, rule = _dcrate_sim)
    
    for j in species:
        b.c[0, j].fix(c0[j])
    
    def _errsq_rule(b):
        return b.errsq == sum((b.c[t, 'a'] - ca_meas[t]) ** 2 + (b.c[t, 'b'] - cb_meas[t]) ** 2 
                   + (b.c[t, 'c'] - cc_meas[t]) ** 2 for t in meas_t) 
    b.errsq_cons = pyo.Constraint(rule=_errsq_rule)
    
    if disctype == 'colloc':
        disc = pyo.TransformationFactory('dae.collocation')
        disc.apply_to(b, nfe=20, ncp=2)
    else:
        disc = pyo.TransformationFactory('dae.finite_difference')
        disc.apply_to(b, nfe=500, scheme = 'BACKWARD')
    
    return m

In [41]:
m = ABC_model(m, data[0], 'colloc', 0)

In [ ]:
sim = dae.Simulator(m, package='scipy') 

In [ ]:
tsim, profiles = sim.simulate(numpoints=100, integrator='vode') 

In [ ]:
fig, ax = plt.subplots()
ax.plot(tsim, profiles[:,0])
ax.plot(tsim, profiles[:,1])
ax.plot(tsim, profiles[:,2])

In [61]:
m = ABC_model(m.b[0], data[0], 'colloc', 0)

In [62]:
m.obj = pyo.Objective(expr = m.b[0].errsq, sense = pyo.minimize)
solver = pyo.SolverFactory('ipopt')
solver.solve(m, tee = True)

Ipopt 3.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      972
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      137

Total number of variables............................:      243
                     variables with only lower bounds:        1
                variables with lower and upper bounds:      122
                     variables with only upper bounds:        0
Total number of equality constraints.................:      241
Total number of inequali

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 241, 'Number of variables': 243, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.05088973045349121}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [65]:
# Estimated parameters
m.k[0](), m.k[1](0)

(2.014379387598257, 0.9944521090927745)

In [ ]:
# Get ca, cb, cc profiles with estimated parameters
ms = ABC_model(data[0], 'colloc')
ms.k1.fix(m.k1())
ms.k2.fix(m.k2())
solver = pyo.SolverFactory('ipopt')
solver.solve(ms, tee = True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(list(ms.time), [ms.ca[t]() for t in ms.time])
ax.plot(list(ms.time), [ms.cb[t]() for t in ms.time])
ax.plot(list(ms.time), [ms.cc[t]() for t in ms.time])
ax.scatter(data[0]['ca_meas'].keys(), data[0]['ca_meas'].values())
ax.scatter(data[0]['cb_meas'].keys(), data[0]['cb_meas'].values())
ax.scatter(data[0]['cc_meas'].keys(), data[0]['cc_meas'].values())

In [ ]:
nlp = PyomoNLP(m)
parm_vars = [m.k1, m.k2]
Hred = hess.getHred(nlp, parm_vars)

In [ ]:
n = len(data[0]['ca_meas']) + len(data[0]['cb_meas']) + len(data[0]['cc_meas'])
p = 2
s2 = m.obj() / (n - p)
print('n:', n, ' p:', p, ' s2:', s2)

In [ ]:
cov = 2 * s2 * np.linalg.inv(Hred)
print('Covariance Matrix')
print(cov)

In [ ]:
parm_sd = np.sqrt(np.diag(cov))
conf_mult = np.sqrt(p * spstat.f.ppf(0.95, p, n - p))
print("conf multiplier:", conf_mult)
conf_int = conf_mult * parm_sd
print("confidence interval delta from nominal")
print(conf_int)